In [5]:
import numpy as np
import cv2 as cv

img_fn_base = "/home/hoseung/Work/tonemap/astroBF/notebooks/natural/StLouisArchMultExpEV"
fn_suffix = ["+4.09.jpeg", "+1.51.jpeg", "-1.82.jpeg" ,"-4.72.jpeg"]
img_list = [cv.imread(img_fn_base+fn) for fn in fn_suffix]
exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)

# Merge muliti-exposure images into one HDR image 
This step won't be necessary for astronomical images. 
So, let's just use a pre-built method.


Available TMOs in OpenCV are:  
cv::TonemapDrago, cv::TonemapMantiuk and cv::TonemapReinhard

In [7]:
# Merge exposures to HDR image
merge_debevec = cv.createMergeDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())

#merge_robertson = cv.createMergeRobertson()
#hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())